In [ ]:
import os
import fsspec
import xarray as xr
from fsspec.implementations.sftp import SFTPFileSystem
import fsspec

Create SFTPFileSystem object to explore SFTP structure

In [ ]:
sftp_host = os.environ["GLEAM_FTP"][7:-1] # host should not have sftp prefix
gleam_creds_sftp = dict(
    username = os.environ["GLEAM_USER"],
    password = os.environ["GLEAM_PASSWORD"],
    port = int(os.environ["GLEAM_PORT"])
    )

In [ ]:
# create link
fs_sftp = SFTPFileSystem(host=sftp_host, **gleam_creds_sftp)

Explore file structure

In [ ]:
fs_sftp.ls("./")

In [ ]:
fs_sftp.ls("./data")

In [ ]:
fs_sftp.ls("./data/v3.7b")

In [ ]:
fs_sftp.ls("./data/v3.7b/daily")

In [ ]:
fs_sftp.ls("./data/v3.7b/daily/2003")

In [ ]:
# see info on file
fs_sftp.info('./data/v3.7b/daily/2003/Et_2003_GLEAM_v3.7b.nc')

Download files

In [ ]:
# this will download the file
# fs_sftp.get('./data/v3.7b/daily/2003/Et_2003_GLEAM_v3.7b.nc', "")

In [ ]:
glob_path = "./data/v3.7b/daily/*/Et*"

In [ ]:
# use glob to get all Et files
Et_file_paths = fs_sftp.glob("./data/v3.7b/daily/*/Et*")
# [print(f) for f in Et_file_paths]

In [ ]:
# set download directory
download_fp = "./download2/"

In [ ]:
fs_sftp.get(rpath=glob_path, lpath=download_fp)

In [ ]:
# download all files from SFTP server
for file in Et_file_paths[0:2]:
    fs_sftp.get(file, download_fp)

In [ ]:
fs_sftp.get?

Create dictionary of environmental variables for GLEAM

In [ ]:
fs_sftp?

In [ ]:
gleam_creds = dict(
    username = os.environ["GLEAM_USER"],
    password = os.environ["GLEAM_PASSWORD"],
    port = int(os.environ["GLEAM_PORT"])
    )

Check that open file objects will work

In [ ]:
years = range(2003, 2023)

for year in years:
    # format file path on SFTP
    filepath = f"data/v3.7b/daily/{year}/Et_{year}_GLEAM_v3.7b.nc"
    # create full URL
    urlpath = os.environ["GLEAM_FTP"] + filepath

    # add to credentials to send to fsspec.open
    gleam_creds["urlpath"] = urlpath

    # create OpenFile object
    file = fsspec.open(**gleam_creds)
    print(file)

## Start creating pangeo-forge-recipe

In [ ]:
import pandas as pd
from pangeo_forge_recipes.patterns import ConcatDim, FilePattern
import apache_beam as beam
from pangeo_forge_recipes.transforms import OpenURLWithFSSpec, OpenWithXarray, StoreToZarr
from tempfile import TemporaryDirectory

Create time range

In [ ]:
dates = pd.date_range("2003", "2005", freq="A")

In [ ]:
time_concat_dim = ConcatDim("time", dates, nitems_per_file=1)
time_concat_dim

In [ ]:
base_url = "sftp://hydras.ugent.be/data/v3.7b/daily/{time:%Y}/Et_{time:%Y}_GLEAM_v3.7b.nc"

In [ ]:
def make_url(time):
    return base_url.format(time=time)

In [ ]:
make_url(dates[-1])

In [ ]:
pattern = FilePattern(make_url, time_concat_dim, fsspec_open_kwargs=gleam_creds)
pattern

In [ ]:
for index, url in pattern.items():
    print(index)
    print(url)
    # Stop after the 3rd filepath (September 3rd, 1981)
    if '19810903' in url:
        break

In [ ]:
td = TemporaryDirectory()
target_path = td.name
target_name = "output.zarr"
target_path

In [ ]:
transforms = (
    beam.Create(pattern.items())
    | OpenURLWithFSSpec()
    | OpenWithXarray(file_type=pattern.file_type)
    | StoreToZarr(
        target_root=target_path,
        store_name=target_name,
        combine_dims=pattern.combine_dim_keys,
    )
)
transforms

Run!

In [ ]:
with beam.Pipeline() as p:
    p | transforms